--- 

<h1 style="color: #D97C00">MÔ TẢ</h1>

---

In [1]:
import random

<h3 style="color:#F9E15B">I. Các thông số tối ưu của GA</h3>

- Kích thước quần thể. (Population Size)
- Số thế hệ. (Generations)
- Tỷ lệ lai ghép. (Crossover Rate)
- Tỷ lệ đột biến. (Mutation Rate)

In [2]:
POPULATION_SIZE = 200  # Kích thước quần thể
GENERATIONS = 200  # Số thế hệ
CROSSOVER_RATE = 0.8  # Tỷ lệ lai ghép
MUTATION_RATE = 0.02  # Tỷ lệ đột biến

<h3 style="color:#F9E15B">II. Các thông số của Knapsack problem</h3>

- Danh sách chứa thông tin về các vật phẩm trong bài toán cái ba lô, bao gồm trọng lượng và giá trị của mỗi vật phẩm.
- Dung lượng tối đa của cái ba lô trong bài toán.
- Danh sách lưu trữ giá trị fitness của các cá thể tốt nhất qua từng thế hệ.

In [11]:
items = [] # Vật phẩm chứa weights và values
max_capacity = 0 # maximum capacity
fitness_history = [] # Chứa lịch sử fitness qua các thế hệ

<h3 style="color:#F9E15B">III. Nhập liệu</h3>

**1. Chuyển đổi chuỗi trọng lượng và giá trị, sức chứa tối đa balo.**

In [4]:
def get_user_input():
    global items, max_capacity
    
    weights_str = input("Nhập danh sách trọng lượng (phân tách bằng dấu phẩy): ")
    values_str = input("Nhập danh sách giá trị (phân tách bằng dấu phẩy): ")
    max_capacity = int(input("Nhập sức chứa tối đa: "))
    
    weights = list(map(int, weights_str.split(",")))
    values = list(map(int, values_str.split(",")))
    items = list(zip(weights, values))
    
    return items, max_capacity

**2. Hiển thị thông tin về các vật phẩm và dung lượng tối đa của cái ba lô.**

In [5]:
def get_info():
    print("\nThông tin bài toán:\n")
    stt = 1
    for weight, value in items:
        print(f"Vật phẩm {stt}: Weight = {weight}, Value = {value}")
        stt += 1
    print(f"\nMaximum capacity = {max_capacity}")

<h3 style="color:#F9E15B">IV. Tính toán giá trị fitness của cá thể dựa trên trọng lượng và giá trị vật phẩm</h3>

- Fitness của mỗi cá thể được tính dựa trên số vật phẩm được chọn và tính tổng value của số vật phẩm được chọn.
- Nếu cá thể có fitness > dung lượng của balo thì fitness sẽ được gán 0.

In [6]:
def fitness(individual):
    total_weight = sum(individual[i] * items[i][0] for i in range(len(items)))
    total_value = sum(individual[i] * items[i][1] for i in range(len(items)))
    return total_value if total_weight <= max_capacity else 0

<h3 style="color:#F9E15B">V. Khởi tạo quần thể</h3>

In [7]:
def initialize_population(num_items):
    return [[random.randint(0, 1) for _ in range(num_items)] for _ in range(POPULATION_SIZE)]

<h3 style="color:#F9E15B">VI. Elitism Selection.</h3>

- Sử dụng chọn lọc ưu tú.
- Chọn cá thể có fitness cao nhất để giữ lại.
- Các cá thể khác được chọn theo thứ tự giảm dần fitness.

In [8]:
def select_population(population):
    sorted_population = sorted(population, key=lambda x: fitness(x), reverse=True)
    return sorted_population[:POPULATION_SIZE]

<h3 style="color:#F9E15B">VII. One-point Crossover.</h3>

- Chọn điểm lai ghép ngẫu nhiên, một điểm để cắt.
- Ghép các phần tử của cả cá thể mẹ và cá thể cha tại điểm lai ghép.

In [9]:
def crossover(parent1, parent2):
    if len(parent1) <= 1 or len(parent2) <= 1:
        return parent1, parent2
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

<h3 style="color:#F9E15B">VIII. Bit-flip mutation.</h3>

- Sử dụng đột biến thông qua cách sử dụng tới thông số tỉ lệ đột biến.
- Duyệt qua từng bit trong cá thể, nếu số random nhỏ hơn thông số tỉ lệ đột biến thì bit bị lật (0 thành 1 hoặc 1 thành 0).

In [ ]:
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            individual[i] = 1 - individual[i]
    return individual

<h3 style="color:#F9E15B">IX. Triển khai thuật toán</h3>

In [ ]:
def genetic_algorithm():
    global fitness_history
    
    population = initialize_population(len(items))
    best_individual = max(population, key=lambda ind: fitness(ind))

    for gen in range(GENERATIONS):
        selected_population = select_population(population)
        
        offspring = []
        for i in range(0, len(selected_population), 2):
            if random.random() < CROSSOVER_RATE and i+1 < len(selected_population):
                child1, child2 = crossover(selected_population[i], selected_population[i+1])
                offspring.extend([child1, child2])
            else:
                offspring.extend([selected_population[i], selected_population[i+1]])

        offspring = [mutate(ind) for ind in offspring]
        
        population = offspring
        current_best = max(population, key=lambda ind: fitness(ind))
        fitness_history.append(fitness(current_best))
        
        if fitness(current_best) > fitness(best_individual):
            best_individual = current_best

    return best_individual, fitness(best_individual)